In [2]:
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import numpy as np
from tensorflow.keras import optimizers

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
!git clone "https://github.com/IndoNLP/indonlu.git"

In [ ]:
df=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/finance dataset - Sheet1.csv")

In [ ]:
df_ext = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/financial_news_mt.csv")

In [ ]:
df_ext_2 = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/financial_news_mt_2.csv")

In [ ]:
df_ext_3 = pd.read_csv("/content/indonlu/dataset/smsa_doc-sentiment-prosa/train_preprocess.tsv", sep = "\t", header = None)

In [ ]:
df = df[['label', 'content']]

In [ ]:
df.head()

In [ ]:
df_ext = df_ext.drop(columns = ["Unnamed: 0", "en_text"])

In [ ]:
df_ext.head()

In [ ]:
df_ext.columns = ["label", "content"]

In [ ]:
df_ext = df_ext[df_ext["label"] != "neutral"]

In [ ]:
df_ext["content"][4]

In [ ]:
label_dict = {'positive':'up', 'negative':'down'}
df_ext["label"] = df_ext["label"].map(label_dict)

In [ ]:
df_ext["label"].value_counts()

In [ ]:
df_ext_2

In [ ]:
df_ext_3.columns = ["content", "label"]

In [ ]:
df_ext_3 = df_ext_3[df_ext_3["label"] != "neutral"]

In [ ]:
df_ext_3

In [ ]:
df = df.iloc[0:83, :]

In [ ]:
label_dict = {'up':'up', 'down':'down'}
df['label'] = df['label'].map(label_dict)

In [ ]:
df_ext_3

In [ ]:
 label_dict = {'positive':'up', 'negative':'down'}

 df_ext_3['label'].map(label_dict)

In [ ]:
df_ext_3['label']

In [ ]:
label_dict = {'positive':'up', 'negative':'down'}

df_ext_3['label'] = df_ext_3['label'].map(label_dict)

In [ ]:
df_ext_3

In [ ]:
df.head()

In [ ]:
df = df.append(df_ext)

In [ ]:
df = df.append(df_ext_2)

In [ ]:
df = df.append(df_ext_3)

In [ ]:
df.sample(20)

In [ ]:
df['text_length'] = df["content"].str.split().str.len()

In [ ]:
df["text_length"][:83].hist(bins=10)

In [ ]:
#df = df[df["text_length"] <= 30]

In [ ]:
df['label'].value_counts()

In [ ]:
test=df[:83]
train=df[83:]

In [ ]:
test.iloc[2]

In [ ]:
test["content"][2]

In [ ]:
train_pos = train[train["label"] == "up"]
train_pos = train_pos.sample(4000 , replace = True)

In [ ]:
train_neg = train[train["label"] == "down"]

In [ ]:
train_all = train_pos.append(train_neg)

In [ ]:
train_all["label"].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import pandas as pd
import time

In [ ]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def cleansing(text):
    text = remove_emojis(text)
    text = re.sub(r'http\S+', '', text)
    text = text.lower()
    word_list = word_tokenize(text)
    word_list = [word for word in word_list if len(word) > 2 and word.isalnum()]
    word_list = [word for word in word_list if string.punctuation not in word]
    text = ' '.join(word_list)
    return text

In [ ]:
import nltk
nltk.download("punkt")

In [ ]:
train['content'] = train['content'].apply(cleansing)
test['content'] = test['content'].apply(cleansing)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("cahya/bert2gpt-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2gpt-indonesian-summarization")

In [ ]:
def machine_summarization(article):
  input_ids = tokenizer.encode(article, return_tensors='pt', 
                               padding=True, truncation=True, 
                               max_length=512, add_special_tokens = True)
  summary_ids = model.generate(input_ids,
              min_length=5,
              max_length=20, 
              early_stopping=True,
              no_repeat_ngram_size=2,
              use_cache=True,
              do_sample = True,
              top_k = 50,
              top_p = 0.95)

  summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  print(summary_text)
  return summary_text


In [ ]:
%%time
summary = machine_summarization('sudah hampir dua tahun kita menghadapi pandemi ternyata dampaknya pun berhasil membuat raksasa terseok awal februari 2022 ini saja kita sudah mendapat berita tentang maskapai penerbangan raksasa indonesia yang dikabarkan kolaps dan akan melakukan phk massal bagaimana fakta dan kronologinya garuda indonesia bangkrut isu garuda indonesia tbk giaa akan bangkrut mencuat saat diketahui bahwa perusahaan tersebut akan menemui kementerian ketenagakerjaan kemnaker ')

In [ ]:
summary

In [ ]:
%%time

summarized_content = [machine_summarization(text) for text in df['content'].values[:83]]

In [ ]:
test["content"] = summarized_content
test["content"] = test["content"].apply(cleansing)

In [ ]:
df.to_csv("dataframe_all.csv")

In [ ]:
test["label"].value_counts()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="<OOV>")

In [ ]:
vocab_size = 30000
embedding_dim = 256
max_length = 50
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [ ]:
label2idx = {'down': 1 ,   'up': 0}
idx2label = {y:x for x,y in label2idx.items()}

In [ ]:
idx2label

In [ ]:
train['label_tf'] = train['label'].map(label2idx)
test['label_tf'] = test['label'].map(label2idx)

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) # di inference cukup load tokenizer

tokenizer.fit_on_texts(df['content'].tolist())
word_index = tokenizer.word_index

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train['content'].tolist())
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding = padding_type, truncating = trunc_type)
testing_sentences = tokenizer.texts_to_sequences(test['content'].tolist())
testing_padded = pad_sequences(testing_sentences, maxlen=max_length, padding = padding_type, truncating = trunc_type)

In [ ]:
training_labels_final = np.array(train['label_tf'].tolist())
testing_labels_final = np.array(test['label_tf'].tolist())

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim , input_length = max_length),
    #tf.keras.layers.Dropout(0.9) ,
   # tf.keras.layers.Conv1D(64, 5, activation='relu'),
    # tf.keras.layers.MaxPooling1D(pool_size = 4 ) , 
   #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)) ,
    #tf.keras.layers.LSTM(16) , 
    tf.keras.layers.Dropout(0.5) , 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(4)),
     #tf.keras.layers.Dense(64 , activation='relu') , 
    tf.keras.layers.Dense(16 , activation='relu'),
    tf.keras.layers.Dropout(0.5) ,
   tf.keras.layers.Dense(256 , activation='relu'),
    tf.keras.layers.Dense( 1 , activation='sigmoid')
])

callbacks = ModelCheckpoint("best_model.h5",
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True)

model.compile(loss='binary_crossentropy', optimizer= optimizers.Adam(learning_rate = 0.001) , 
              metrics=['accuracy']) 
model.summary()
# cross entropy = negative log likelihood

In [ ]:
testing_padded

In [ ]:
testing_labels_final

In [ ]:
num_epochs = 10
history = model.fit(train_padded, training_labels_final, batch_size = 128 ,  
                    epochs=num_epochs, validation_data=(testing_padded, testing_labels_final),
              callbacks=[callbacks])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

In [ ]:
model.evaluate(testing_padded, testing_labels_final, batch_size = 1)

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
"""model.save('model.h5')"""

In [ ]:
"""if __name__ == '__main__':
    model.save("model.h5")"""

In [ ]:
"""if __name__ == '__main__':
    model.save("model.pkl")"""

In [ ]:
"""import io, json
tokenizer_json = tokenizer.to_json()
with io.open('saved_model/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))"""

In [ ]:
"""with open('saved_model/label2idx.json', 'w') as fp:
    json.dump(label2idx, fp)
with open('saved_model/idx2label.json', 'w') as fp:
    json.dump(idx2label, fp)"""

In [ ]:
#def tf_prediction(input_text, model, tokenizer):
   # input_sequence = tokenizer.texts_to_sequences([input_text])
   # input_padded = pad_sequences(input_sequence, maxlen=50, padding = 'post', truncating = 'post')
   # predicted = model(input_padded)
   # print(predicted)
   # label_predicted = np.argmax(predicted[0])
   # return label_predicted, predicted[0][label_predicted]

#def tf_prediction_batch(list_text, model, tokenizer):
  #  input_sequence = tokenizer.texts_to_sequences(list_text)
   # input_sequence = pad_sequences(input_sequence, maxlen=50, padding = 'post', truncating = 'post')
   # print(input_sequence.shape)
   # predicted = model(input_sequence)
   # label_predicted = np.argmax(predicted, axis = 1)
   # return predicted, label_predicted

In [ ]:
#output_label = tf_prediction('krisis moneter indonesia', model, tokenizer )

In [ ]:
#output_label

In [ ]:
"""idx2label = {y:x for x,y in label2idx.items()}"""

In [ ]:
"""idx2label[output_label[0]]"""

In [ ]:
!cp "/content/best_model.h5" "/content/gdrive/MyDrive/Flask NLP"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp "/content/tokenizer.pickle" "/content/gdrive/MyDrive/Flask NLP"